The goal of this analysis is to get an intuition for how genetic variant rarity changes with (predicted) genic consequence. 

This notebook will compute count tables of variant rarity category by ensembl predicted consequence, plus phylop & roulette scores (sum and sum of squares). 

## Import relevant libraries

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import json

## create the spark session.

In [3]:
spark = SparkSession.builder \
    .appName("purifying_selection") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/08 12:40:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load variants

In [4]:
#real data

df = spark.read \
    .option("comment", "#") \
    .option("delimiter", "\t") \
    .option("header", "true") \
    .csv("/gpfs/gibbs/pi/reilly/VariantEffects/scripts/noon_data/2.3.add_transposons/*.csv.gz/*.csv.gz")

#toy data

#df=spark.read \
#    .option("delimiter","\t") \
#    .option("header","true") \
#    .csv("toy_with_missing_vep.tsv")
#OR
#    .csv("toy.tsv")



24/08/08 12:40:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Note that since we're tapping off variants at the 2.3 add transposon step, we're missing:
- 2.5 filter : so we don't filter out exonic variants (this is desireable)
- 3.0 pleio_and_filter : so we haven't dropped MAF_OR_AC_IS_ZERO (which is performed below)
- 3.5 add_tf : (no great loss)
- 3.6 remove non-snp (which we do below)

## Filter out non-SNP variants

In [5]:
df= df.filter(
     df.REF.isin("A", "T", "C", "G") & df.ALT.isin("A", "T", "C", "G")
)

# Filter out `MAF_OR_AC_IS_ZERO`

In [6]:
df=df.filter(F.col("category")!="MAF_OR_AC_IS_ZERO")

## Count occurances of each consequence code in each vep string.

First, get a list of consequences for each variant. This is a little involved, because of the many layers we have to trawl through:

![schema](./info_field.drawio.png)

In [7]:
#semicolon split
df=df.withColumn("info_split",F.split(df["INFO"],";"))
df=df.withColumn("vep_alone",F.expr("filter(info_split, x -> x LIKE 'vep=%')[0]"))

In [8]:
#comma split
df=df.withColumn("vep_split",F.split(df["vep_alone"],","))

#pipe split & grab first element. 
df = df.withColumn(
    "extracted_codes",
    F.transform(F.col("vep_split"), lambda x: F.split(x, "\\|")[1])
)

In [9]:
#break up anpersand-ligated conseqence codes
df = df.withColumn(
    "consq_codes",
    F.expr(
        "flatten(transform(extracted_codes, x -> IF(x IS NOT NULL, split(x, '&'), array())))"
    )
)
#Note the IS NOT NULL : the presence of comma-delimited protein information can bork a subset of the entries, which we need to skip

In [10]:
df

DataFrame[CHROM: string, POS: string, REF: string, ALT: string, ID: string, QUAL: string, FILTER: string, INFO: string, K562__ref: string, HepG2__ref: string, SKNSH__ref: string, K562__alt: string, HepG2__alt: string, SKNSH__alt: string, K562__skew: string, HepG2__skew: string, SKNSH__skew: string, AC: string, AN: string, AF: string, cadd_phred: string, is_in_dELS: string, is_in_CA: string, is_in_pELS: string, is_in_CA-H3K4me3: string, is_in_CA-CTCF: string, is_in_PLS: string, is_in_TF: string, is_in_CA-TF: string, P_ANNO: string, mean_ref: string, mean_skew: string, MAF: string, category: string, roulette_PN: string, roulette_MR: string, roulette_MG: string, in_rep: string, info_split: array<string>, vep_alone: string, vep_split: array<string>, extracted_codes: array<string>, consq_codes: array<string>]

In [9]:
###
#with open("dump", "w") as file:
#    for i in df.where(F.col("consq_codes").isNull()).take(10):
#    #for i in df.take(1000):
#        file.write(str(i["vep_alone"])+str(i["extracted_codes"])+"\t"+str(i["consq_codes"])+"\n")

In [11]:
#Some variants might have no predicted consequences. We will use `absent`
df=df.withColumn("consq_codes", F.when(F.col("consq_codes").isNull(), F.array(F.lit("absent"))).otherwise(F.col("consq_codes")))

Next, compute the worst consequence code for each var.

I've retrieved consequences from [here](https://useast.ensembl.org/info/genome/variation/prediction/predicted_data.html) on 2024-06-10. 

In [12]:
#This order is taken from the website linked above, which states that 
#the codes are shown in order of severity (though it admits this is subjective)
#I've assigned numbers, where the smaller the more severe

consq_code_lut = {"transcript_ablation":0, 
                  "splice_acceptor_variant":1, 
                  "splice_donor_variant":2, 
                  "stop_gained":3, 
                  "frameshift_variant":4, 
                  "stop_lost":5, 
                  "start_lost":6, 
                  "transcript_amplification":7,
                  "feature_elongation":8,
                  "feature_truncation":9,
                  "inframe_insertion":10,
                  "inframe_deletion":11,
                  "missense_variant":12,
                  "protein_altering_variant":13,
                  "splice_donor_5th_base_variant":14,
                  "splice_region_variant":15,
                  "splice_donor_region_variant":16,
                  "splice_polypyrimidine_tract_variant":17,
                  "incomplete_terminal_codon_variant":18,
                  "start_retained_variant":19,
                  "stop_retained_variant":20,
                  "synonymous_variant":21,
                  "coding_sequence_variant":22,
                  "mature_miRNA_variant":23,
                  "5_prime_UTR_variant":24,
                  "3_prime_UTR_variant":25,
                  "non_coding_transcript_exon_variant":26,
                  "intron_variant":27,
                  "NMD_transcript_variant":28,
                  "non_coding_transcript_variant":29,
                  "coding_transcript_variant":30,
                  "upstream_gene_variant":31,
                  "downstream_gene_variant":32,
                  "TFBS_ablation":33,
                  "TFBS_amplification":34,
                  "TF_binding_site_variant":35,
                  "regulatory_region_ablation":36,
                  "regulatory_region_amplification":37,
                  "regulatory_region_variant":38,
                  "intergenic_variant":39,
                  "sequence_variant":40,
                  "absent":41
                 }

In [18]:
# break into separate columns for individual enumeration


for code in list(consq_code_lut.keys()):
    df=df.withColumn(code,F.when(F.array_contains(F.col("consq_codes"),code),F.lit(True)).otherwise(F.lit(False)))

#list(possible_consequence_codes)

In [20]:
#import pandas as pd
#
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    display(df.limit(5).toPandas())

24/08/08 12:56:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,CHROM,POS,REF,ALT,ID,QUAL,FILTER,INFO,K562__ref,HepG2__ref,SKNSH__ref,K562__alt,HepG2__alt,SKNSH__alt,K562__skew,HepG2__skew,SKNSH__skew,AC,AN,AF,cadd_phred,is_in_dELS,is_in_CA,is_in_pELS,is_in_CA-H3K4me3,is_in_CA-CTCF,is_in_PLS,is_in_TF,is_in_CA-TF,P_ANNO,mean_ref,mean_skew,MAF,category,roulette_PN,roulette_MR,roulette_MG,in_rep,info_split,vep_alone,vep_split,extracted_codes,consq_codes,transcript_ablation,splice_acceptor_variant,splice_donor_variant,stop_gained,frameshift_variant,stop_lost,start_lost,transcript_amplification,feature_elongation,feature_truncation,inframe_insertion,inframe_deletion,missense_variant,protein_altering_variant,splice_donor_5th_base_variant,splice_region_variant,splice_donor_region_variant,splice_polypyrimidine_tract_variant,incomplete_terminal_codon_variant,start_retained_variant,stop_retained_variant,synonymous_variant,coding_sequence_variant,mature_miRNA_variant,5_prime_UTR_variant,3_prime_UTR_variant,non_coding_transcript_exon_variant,intron_variant,NMD_transcript_variant,non_coding_transcript_variant,coding_transcript_variant,upstream_gene_variant,downstream_gene_variant,TFBS_ablation,TFBS_amplification,TF_binding_site_variant,regulatory_region_ablation,regulatory_region_amplification,regulatory_region_variant,intergenic_variant,sequence_variant,absent
0,chr2,54064,G,A,rs994438331,.,PASS,K562__ref=-0.060518835;HepG2__ref=-0.11972282;...,-0.060518835,-0.11972282,-0.31378546,-0.061227016,-0.13887358,-0.30973908,-7.081867E-4,-0.019150767,0.00404637,2,152162,1.31439E-5,1.57,false,false,false,false,false,false,false,false,-0.6,-0.16467571258544922,-0.0052708617101113004,1.31439E-5,ULTRARARE,AGGCT,0.151,0.154,false,"[K562__ref=-0.060518835, HepG2__ref=-0.1197228...",vep=A|intergenic_variant|MODIFIER|||Intergenic...,[vep=A|intergenic_variant|MODIFIER|||Intergeni...,[intergenic_variant],[intergenic_variant],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,chr2,90037,A,G,rs77219298,.,PASS,K562__ref=0.6132817;HepG2__ref=0.60334325;SKNS...,0.6132817,0.60334325,1.037919,0.6745551,0.64999133,1.084241,0.061273456,0.04664812,0.046321955,6130,152140,0.0402918,8.437,false,false,false,false,false,false,false,false,0.418,0.7515146732330322,0.05141450961430868,0.0402918,COMMON,CTATA,0.431,0.211,false,"[K562__ref=0.6132817, HepG2__ref=0.60334325, S...",vep=G|intergenic_variant|MODIFIER|||Intergenic...,[vep=G|intergenic_variant|MODIFIER|||Intergeni...,[intergenic_variant],[intergenic_variant],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
2,chr2,127009,G,A,rs1382759750,.,PASS,K562__ref=0.28705546;HepG2__ref=0.62864524;SKN...,0.28705546,0.62864524,0.61934525,0.26176712,0.5995152,0.5896472,-0.025288366,-0.02913005,-0.029697992,4,148076,2.70132E-5,3.898,false,true,false,false,false,false,false,false,0.0,0.5116819540659586,-0.028038802246252697,2.70132E-5,ULTRARARE,GGGGA,0.223,0.147,true,"[K562__ref=0.28705546, HepG2__ref=0.62864524, ...",vep=A|intergenic_variant|MODIFIER|||Intergenic...,[vep=A|intergenic_variant|MODIFIER|||Intergeni...,[intergenic_variant],[intergenic_variant],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
3,chr2,216454,T,C,rs1268178481,.,PASS,K562__ref=-0.051730506;HepG2__ref=0.043524552;...,-0.051730506,0.043524552,0.14609466,-0.061243348,0.035201576,0.13232395,-0.009512845,-0.008322978,-0.013770727,1,152222,6.56935E-6,6.16,false,false,false,false,false,false,true,false,0.771,0.04596290489037832,-0

In [ ]:
# compute worst consequence code for bulk enumeration

In [12]:
lookup_broadcast = spark.sparkContext.broadcast(consq_code_lut)

reverse_consequence_code_lut= {value: key for key, value in consq_code_lut.items()}

#lookup_broadcast_reverse = spark.sparkContext.broadcast(reverse_consequence_code_lut)

In [13]:
def lookup_transform(inp):
    #turns a list of consequence codes into a list of severity ints
    lookup=lookup_broadcast.value
    return [lookup.get(item) for item in inp]

def lookup_transform_reverse(inp):
    #turns a SINGLE severity int into a consequence code
    return reverse_consequence_code_lut.get(inp,"ERR")

#register the UDFs
lookup_transform_udf = F.udf(lookup_transform, returnType=T.ArrayType(T.IntegerType()))

lookup_transform_reverse_udf = F.udf(lookup_transform_reverse, returnType=T.StringType())

In [14]:
#Apply the lookup UDF to convert string consequence codes to severity ints
df=df.withColumn("consq_numeric",lookup_transform_udf(df["consq_codes"]))

In [15]:
#get the worst severity score for each variant.
df=df.withColumn("min_consq_numeric", F.array_min(df["consq_numeric"]))

In [16]:
#convert minimum consequence code back to string
df=df.withColumn("worst_consq_string",
              lookup_transform_reverse_udf(df["min_consq_numeric"])
             )

In [17]:
#manual verification
#import pandas as pd
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    display(df.limit(3).toPandas())
#df.limit(3).toPandas()["consq_codes"].to_list()

In [ ]:
df=df.withColumn("phylop_significant",F.col("P_ANNO")>=2.27)

In [18]:
#count 
counts=df.groupBy("category","worst_consq_string","phylop_significant",*list(consq_code_lut.keys())).agg(
    
    F.sum("P_ANNO").alias("sum_phylop"),
    F.sum(F.col("P_ANNO") * F.col("P_ANNO")).alias("sum_of_squared_phylop"),
    
    F.sum("roulette_MR").alias("sum_roulette_MR"),
    F.sum(F.col("roulette_MR") * F.col("roulette_MR")).alias("sum_of_squared_roulette_MR"),
    
    
    F.count("*").alias("count")  # Count of elements in each group
)

Dump to disc

In [70]:
#COMMENTED OUT FOR DEBUG
counts.coalesce(1).write.csv("counts_broken.csv", mode="overwrite", header=True)

#array_columns = [field.name for field in df.schema.fields if isinstance(field.dataType, T.ArrayType)]

## Cast ARRAY<...> columns to strings
#for col_name in array_columns:
#    df = df.withColumn(col_name, F.concat_ws("-", F.col(col_name)))



In [19]:
#collected_data=df.where(df.worst_consq_string == "absent").limit(5).collect()
##collected_data=df.limit(5).collect()
##with open("dump_normal.tsv", "w") as file:
#with open("dump.tsv", "w") as file:
#    # Write the header
#    header = "\t".join(df.columns)
#    file.write(header + "\n")
#    
#    # Write the data rows
#    for row in collected_data:
#        line = "\t".join(map(str, row))
#        file.write(line + "\n")

24/07/26 11:50:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
ERROR:root:KeyboardInterrupt while sending command.              (0 + 10) / 100]
Traceback (most recent call last):
  File "/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [52]:
#df.where(df.worst_consq_string == "absent").limit(5).write.csv("dumpABSENT.csv", mode="overwrite", header=True)

Traceback (most recent call last):                              (1 + 10) / 2258]
  File "/vast/palmer/home.mccleary/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 193, in manager
  File "/vast/palmer/home.mccleary/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/vast/palmer/home.mccleary/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1291, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/vast/palmer/home.mccleary/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError
24/07/26 09:44:36 ERROR PythonUDFRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/va

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_3768885/4213338293.py", line 4, in lookup_transform
TypeError: 'NoneType' object is not iterable


24/07/26 09:44:37 WARN TaskSetManager: Lost task 0.0 in stage 12.0 (TID 17228) (r814u23n03.mccleary.ycrc.yale.edu executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 9 in stage 12.0 failed 1 times, most recent failure: Lost task 9.0 in stage 12.0 (TID 17237) (r814u23n03.mccleary.ycrc.yale.edu executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_3768885/4213338293.py", line 4, in lookup_transform
TypeError: 'NoneType' object is not iterable

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.ha